# SIMULAR GRIPPER FLOTANTE EN GAZEBO
## Iniciando la simulacion de Gazebo

Para comenzar a visualizar e interactuar con el brazo robótico UR10 en un entorno simulado, necesitas iniciar el entorno ROS utilizando RViz. Por favor, sigue estos pasos:

1. Abre una Nueva Terminal: Primero, necesitas abrir una nueva ventana de terminal en tu sistema (Ctrl + Alt + T).

2. Ejecuta el Comando de Lanzamiento: En la terminal recién abierta, ejecuta el siguiente comando:


``` shellscript
user@pc:~$ roslaunch manipulacion_pkg gripper_gazebo.launch tipo_gripper:=jaco objeto:=mustard_bottle


```

# Ejemplo: Transformacion de coordenadas para pose gripper flotante
En este ejemplo, nos centraremos en modificar la pose de un gripper flotante en Gazebo, basándonos en su posición relativa a un objeto situado en el entorno. A través de este ejemplo, aprenderemos a aplicar transformaciones de coordenadas para ajustar la posición y orientación del gripper en función de la ubicación de un objeto específico. Para hacer esto, necesitamos ejecutar los siguientes pasos:

1. **Inicializar el Nodo ROS**: Comenzamos inicializando un nodo ROS llamado 'sistemas_coordenadas_y_transformaciones'. Este nodo nos permitirá comunicarnos con otras partes del sistema ROS. La condición if not rospy.get_node_uri() asegura que solo inicializamos el nodo si aún no ha sido inicializado.
2. **Creación de una Instancia de Simulación del Gripper Flotante**:  Se crea una instancia de SimulacionGripperFlotante de la biblioteca manipulacion_lib. Esta instancia se utiliza para fijar y obtener la pose del gripper flotante y los objetos del entorno en la simulación de Gazebo. Se especifica el nombre del gripper en Gazebo, en este caso, "gripper".
3. **Obtener pose del objeto con respecto al sistema de referencia global**:  Se utiliza el método obtener_pose_objeto para obtener la pose (como un Frame) del objeto (en este caso, 'mustard_bottle') con respecto al sistema de coordenadas global (world).
4. **Fijar pose relativa del gripper con respecto al objeto**: El gripper estará a 0.2 metros en el eje X y 0.1 metros en el eje Z del sistema de coordenadas del objeto sin un cambio en la orientacion con respecto al sistema del objeto.
5. **Transformación de Coordenadas**: Se calcula la pose del gripper en el sistema de coordenadas global (pose_gripper_world) multiplicando la pose del objeto en el sistema global (pose_objeto_world) por la pose del gripper con respecto al objeto (pose_gripper_objeto).

6. **Fijar la Pose del Gripper en Gazebo**: Finalmente, se fija la pose calculada del gripper en la simulación de Gazebo utilizando el método fijar_pose_gripper de la instancia simulacion_gripper_flotante.

![Gripper_pose_relativa_objeto](../images/gripper_flotante_pose_relativa.png)



In [1]:
import manipulacion_lib # Importa la biblioteca para manipulación de objetos y grippers en simulación
import rospy # Importa rospy, necesario para la comunicación con el sistema ROS
import PyKDL # Importa PyKDL para las transformaciones entre sistemas de referencia

# Inicializar el nodo de ROS si aún no se ha inicializado
if not rospy.get_node_uri():
    rospy.init_node('gripper_flotante_gazebo', anonymous=True, log_level=rospy.WARN)

# Creación de una instancia para controlar un gripper flotante en Gazebo
simulacion_gripper_flotante = manipulacion_lib.SimulacionGripperFlotante(nombre_gripper_gazebo="gripper")
# Esta instancia permite interactuar con un gripper en la simulación de Gazebo, especificando su nombre
# Independientemente del gripper asignado para la práctica, se utiliza el nombre "gripper" para referenciarlo
# en la simulación de Gazebo.


# Obtener la pose (posición y orientación) de un objeto específico en el Gazebo
# Se obtiene la pose del objeto 'mustard_bottle' con respecto al sistema de referencia global (world)
pose_objeto_world = simulacion_gripper_flotante.obtener_pose_objeto(nombre_objeto_gazebo='driller_small')

print("Pose objeto con respecto a world")
print(pose_objeto_world)

# Fijamos una pose relativa del gripper con respecto al objeto
# Pose del gripper con respecto al objeto, especificando una traslación de 0.2m en X y 0.1m en Z
pose_gripper_objeto = PyKDL.Frame(PyKDL.Rotation.Quaternion(1.0, 0, 0.0, 0.0), PyKDL.Vector(0.2, 0, 0.1))

print("Pose gripper con respecto a objeto")
print(pose_gripper_objeto)

# Calcular la pose del gripper en el sistema de referencia global a partir de su pose relativa al objeto
# Se realiza una transformación de coordenadas para obtener la pose del gripper en el sistema global
pose_gripper_world = pose_objeto_world * pose_gripper_objeto
print("Pose gripper con respecto a world: ")
print( pose_gripper_world)
# Fijar la pose del gripper en la simulación de Gazebo
simulacion_gripper_flotante.fijar_pose_gripper(pose_gripper_world=pose_gripper_world)
# Actualiza la posición y orientación del gripper en Gazebo según la pose calculada

Pose objeto con respecto a world
[[    0.999036,   0.0104216,    0.042654;
   -0.0103477,    0.999945, -0.00195206;
    -0.042672,   0.0015088,    0.999088]
[    0.304928,    0.200294,    0.082963]]
Pose gripper con respecto a objeto
[[           1,           0,           0;
            0,          -1,           0;
            0,           0,          -1]
[         0.2,           0,         0.1]]
Pose gripper con respecto a world: 
[[    0.999036,  -0.0104216,   -0.042654;
   -0.0103477,   -0.999945,  0.00195206;
    -0.042672,  -0.0015088,   -0.999088]
[       0.509,    0.198029,    0.174337]]


True

# Ejemplo 2 : Modificación de la Pose del Gripper Flotante en Gazebo

En este ejercicio, comenzaremos fijando una pose aleatoria para el gripper flotante en la simulación de Gazebo. A continuación, calcularemos una nueva pose que esté desplazada en el eje X local del gripper por un offset determinado. Esta nueva pose también se fijará en la simulación para observar el cambio.

Pasos del Ejemplo:

1. **Fijar una Pose Aleatoria del Gripper:** Inicialmente, colocaremos el gripper flotante en una pose aleatoria dentro de la simulación de Gazebo.
2. **Calcular la Nueva Pose**: Basándonos en la pose actual del gripper, calcularemos una nueva pose desplazada un offset en el eje X local del gripper.  La imagen muestra los ejes locales del gripper, con el eje X representado en rojo. Este visual es crucial para comprender la dirección del desplazamiento aplicado. 

3. **Fijar la nueva pose del gripper en Gazebo**: La pose recalculada será fijada en la simulación para observar el cambio de la pose del gripper.
![EjesLocalesJacoHand](../images/ejes_locales_jaco_hand.png)


## ¡IMPORTANTE!:
 Para crear la impresión de que el gripper se está moviendo a lo largo de una línea recta entre las dos poses, se podría realizar una interpolación entre ambas poses. Este proceso implicaría ajustar gradualmente la posición del gripper a lo largo del tiempo, dando lugar a un movimiento suave y continuo. Se anima a los estudiantes a implementar esta interpolación por su cuenta, aplicando los conceptos aprendidos sobre transformaciones de coordenadas y manipulación de poses en un entorno de simulación.


In [2]:
import manipulacion_lib
import rospy
import numpy as np


# Inicializar el nodo de ROS si aún no se ha inicializado
if not rospy.get_node_uri():
    rospy.init_node('gripper_flotante_gazebo', anonymous=True, log_level=rospy.WARN)

# Creación de una instancia para controlar el gripper flotante en la simulación de Gazebo
simulacion_gripper_flotante = manipulacion_lib.SimulacionGripperFlotante(nombre_gripper_gazebo="gripper")
# Generación de una pose aleatoria para el gripper
# Se generan tres números aleatorios uniformes para la posición en el espacio, limitados entre -1 y 1 para x e y,
# y entre 0.5 y 1 para z.
posicion_gripper_world = [np.random.uniform(-1, 1), np.random.uniform(-1, 1), np.random.uniform(0.5, 1)]
# Se generan cuatro números aleatorios para la orientación (como cuaternión), cada uno entre -1 y 1.
orientacion_gripper_world = [np.random.uniform(-1, 1), np.random.uniform(-1, 1), np.random.uniform(-1, 1), np.random.uniform(-1, 1)]
# Normalización del cuaternión para garantizar que representa una rotación válida.
orientacion_gripper_world = orientacion_gripper_world / np.linalg.norm(orientacion_gripper_world)
# Construcción de la pose del gripper con los valores aleatorios de posición y orientación.
pose_gripper_world = PyKDL.Frame(PyKDL.Rotation.Quaternion(orientacion_gripper_world[0], orientacion_gripper_world[1],
                                                           orientacion_gripper_world[2], orientacion_gripper_world[3]),
                                 PyKDL.Vector(posicion_gripper_world[0], posicion_gripper_world[1], posicion_gripper_world[2]))

# Fijar la pose aleatoria del gripper en la simulación de Gazebo.
simulacion_gripper_flotante.fijar_pose_gripper(pose_gripper_world=pose_gripper_world)



True

In [3]:
offset = 0.2 # Definimos un offset de 0.2 metros

# Creamos una nueva pose de tipo Frame de PyKDL que no altera la rotación (identidad)
# y desplaza el gripper 0.2 metros a lo largo del eje X local.
pose_nueva_gripper = PyKDL.Frame(PyKDL.Rotation(), PyKDL.Vector(offset, 0, 0 ))

# Multiplicamos la pose actual del gripper en el mundo por la transformación que acabamos
# de crear para obtener la nueva pose en el sistema de coordenadas del mundo.
pose_nueva_world = pose_gripper_world*pose_nueva_gripper

# Fijamos la nueva pose del gripper en la simulación de Gazebo.
simulacion_gripper_flotante.fijar_pose_gripper(pose_gripper_world=pose_nueva_world)


True

# Detectar colisiones entre el gripper y el entorno

In [4]:
from manipulacion_lib import Obstaculo, DetectorColisionesGripperFlotante
obstaculo_rojo = Obstaculo('cubo', [0.75,0,0,1,0,0,0], [0.3,1,0.6], 'obstaculo')
suelo = Obstaculo('cubo', [0,0,0,1,0,0,0], [4,4,0.1], 'suelo')
obstaculos = [obstaculo_rojo, suelo]

detector_colisiones = DetectorColisionesGripperFlotante('schunk', obstaculos)
# Obtener la matriz de rotación del frame
rotation = pose_gripper_world.M

# Convertir la matriz de rotación a cuaternión
qx, qy, qz, qw = rotation.GetQuaternion()
pose_gripper = [pose_gripper_world.p.x(), pose_gripper_world.p.y(), pose_gripper_world.p.z(),
                qx, qy, qz, qw]
print("Colisiona en la pose actual:", detector_colisiones.hay_colision(pose_gripper))

/manipulacion_ws/src/manipulacion_pkg/urdf/schunk.urdf


IndexError: tuple index out of range


#  MOVER ARTICULACIONES DEL GRIPPER
1. **Configuración Inicial del Gripper**: Utilizamos la función `configurar_gripper()` para cargar configuraciones específicas del gripper, incluyendo nombres de articulaciones y posiciones iniciales, desde un archivo YAML. 
2. **Abrir gripper**:  `abrir_gripper()` para abrir el gripper en Gazebo.
3. **Obtener el tipo de gripper**: `get_tipo_gripper()` para saber con que gripper estamos trabajando.
4. **Configuración de Posiciones**: Basado en el tipo, asignamos posiciones específicas a las articulaciones.
5. **Aplicar Configuración**: Con `set_posicion_articulaciones()`, ajustamos las articulaciones del gripper. Cada tipo de gripper tiene un número distinto de articulaciones, lo que requiere una configuración específica para su correcto funcionamiento.

## ¡IMPORTANTE!
Como parte de vuestro trabajo con grippers en simulaciones, es crucial utilizar las posiciones de las articulaciones (dofs) específicas para cada pose de agarre que hayáis obtenido mediante el uso de GraspIt! y guardado previamente en un archivo. 


In [15]:
# Configurar el gripper basado en el archivo YAML específico para su tipo.
# Esto incluye cargar las configuraciones iniciales como nombres de articulaciones y posiciones predeterminadas.
simulacion_gripper_flotante.configurar_gripper()

# Abrir completamente el gripper antes de intentar cualquier operación de agarre.
# Es una práctica común asegurarse de que el gripper no esté restringido o en una posición que podría interferir con el objeto a agarrar.
simulacion_gripper_flotante.abrir_gripper()

# Pequeña pausa para asegurar que la acción de abrir se ha completado.
rospy.sleep(1)

# Obtener el tipo de gripper actual para aplicar la configuración de posiciones de articulaciones correcta.
tipo_gripper = simulacion_gripper_flotante.get_tipo_gripper()

# Basado en el tipo de gripper, se definen las posiciones específicas de las articulaciones.
# Estas posiciones deben ser determinadas experimentalmente o calculadas para lograr el agarre deseado.
if tipo_gripper == 'jaco':
  posicion_articulaciones = [-0.6,-0.6,-0.6]  # Posiciones para el gripper Jaco.
elif tipo_gripper == 'schunk':
  posicion_articulaciones = [0, 0, 0, 0, 0, 0, 0]  # Posiciones para el gripper Schunk.
elif tipo_gripper == 'robotiq':
  posicion_articulaciones = [0.4, 0.4, 0.4, 0, 0.4, 0.4, 0.4, 0, 0.4, 0.4, 0.4]  # Posiciones para el gripper Robotiq.

# Aplicar las posiciones definidas a las articulaciones del gripper.
# Este paso es crucial para controlar el estado del gripper y realizar agarres efectivos en la simulación.
simulacion_gripper_flotante.set_posicion_articulaciones(posicion_articulaciones)


Gripper posicion articulaciones:  [0.0, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5]


KeyboardInterrupt: 